In [1]:
#import desired packages
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
import sys
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten
# library for freezing the graph
from tensorflow.python.tools import freeze_graph
# library for optmising inference
from tensorflow.python.tools import optimize_for_inference_lib

Using TensorFlow backend.


In [2]:
from sklearn.datasets import load_iris

In [3]:
iris = load_iris()

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(iris.data,iris.target,test_size=0.2)
Y_train = keras.utils.to_categorical(Y_train, 3)
Y_test = keras.utils.to_categorical(Y_test, 3)

In [5]:
#placeholders and variables. input has 4 features and output has 3 classes
x=tf.placeholder(tf.float32,shape=[None,4] , name="Input")
y=tf.placeholder(tf.float32,shape=[None, 3])
#weight and bias
W=tf.Variable(tf.zeros([4,3]))
b=tf.Variable(tf.zeros([3]))

# model 
#softmax function for multiclass classification
pred = tf.nn.softmax((tf.matmul(x, W) + b),name="Output")

# Cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

# Optimiser 
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [6]:
#session parameters
sess = tf.InteractiveSession()
#initialising variables
init = tf.global_variables_initializer()
sess.run(init)
#number of interations
epoch=2000

In [7]:
#Training 
for step in range(epoch):
    _,loss=sess.run([optimizer,cost], feed_dict={x: X_train, y:Y_train})
    if step%500==0 :
        print (loss)

1.0986121
0.6322373
0.59874046
0.58749604


In [8]:
# Find the maximum in y for each data point , because that is the digit that I am going to predict
predictions = tf.arg_max(pred,1)
correct_labels = tf.arg_max(y,1)
correct_predictions = tf.equal(predictions,correct_labels)

predictions_evaluated,correct_pred = sess.run([predictions,correct_predictions],feed_dict={x:X_test,y:Y_test})
# feed_dict only need x as predictions depend only upon x and not on any other parameter
predictions_evaluated,correct_pred # False mean wrong and True means right
print("Accuracy is :" ,correct_pred.sum()/len(Y_test) * 100,"%")
print(correct_pred)

Instructions for updating:
Use `argmax` instead
Accuracy is : 96.66666666666667 %
[ True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [9]:
# grabbing the default graph
g = tf.get_default_graph()

# every operations in our graph
# [op.name for op in g.get_operations()]

In [10]:
saver = tf.train.Saver()
model_directory='model_files/'
if not os.path.exists(model_directory):
        os.makedirs(model_directory)
#saving the graph
tf.train.write_graph(K.get_session().graph_def, model_directory, 'savegraph.pbtxt')

# saving the values of weights and other parameters of the model
saver.save(K.get_session(), 'model_files/model.ckpt')

'model_files/model.ckpt'

In [11]:
# Freeze the graph
MODEL_NAME = 'iris'
input_graph_path = 'model_files/savegraph.pbtxt'
checkpoint_path = 'model_files/model.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "Output"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'model_files/frozen_model_'+MODEL_NAME+'.pb'
output_optimized_graph_name = 'model_files/optimized_inference_model_'+MODEL_NAME+'.pb'
clear_devices = True

In [12]:
freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from model_files/model.ckpt
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [14]:
output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        sess.graph_def,
        ["Input"], # an array of the input node(s)
        ["Output"], # an array of output nodes
        tf.float32.as_datatype_enum)

In [15]:
with tf.gfile.GFile(output_optimized_graph_name, "wb") as f:
    f.write(output_graph_def.SerializeToString())

In [16]:
g1 = tf.GraphDef()
##checking frozen graph
g1.ParseFromString(open("model_files/frozen_model_iris.pb", 'rb').read())
# g1

565

In [17]:
df=pd.DataFrame(iris.target,columns=['Result'])

In [25]:
pd.DataFrame(iris.data).tail()

,0,1,2,3
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [26]:
pd.DataFrame(iris.target).tail()

,0
145,2
146,2
147,2
148,2
149,2
